# Сохранение и загрузка моделей

В этом блокноте показано, как сохранять и загружать модели с помощью PyTorch. Это важно, поскольку вам часто захочется загружать ранее обученные модели для использования в прогнозах или для продолжения обучения на новых данных.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import helper
import fc_model

In [ ]:
# Определим трансформацию для нормализации данных
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Загружаем обучающие данные
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Загружаем тестовые данные
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Отобразим одно из изображений.

In [ ]:
image, label = next(iter(trainloader))
helper.imshow(image[0,:]);

# Обучение сети

Чтобы сделать изложение более кратким, архитектура модели и код обучения перенесены из последней части в файл `fc_model`. Импортируя его, мы можем легко создать полносвязную сеть с помощью `fc_model.Network` и обучить сеть, используя `fc_model.train`. Будет использоваться эта модель (после её обучения), чтобы продемонстрировать, как можно сохранять и загружать модели.

In [ ]:
# Создаем сеть, определяем функцию потерь и оптимизатор
model = fc_model.Network(784, 10, [512, 256, 128])
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
fc_model.train(model, trainloader, testloader, criterion, optimizer, epochs=2)

## Сохранение и загрузка сетей

Как вы можете понять, нецелесообразно обучать сеть каждый раз, когда вам нужно её использовать. Вместо этого мы можем сохранить обученные сети, а затем загрузить их позже, чтобы продолжить обучение или использовать для прогнозов.

Параметры сетей в PyTorch хранятся в `state_dict` модели. Мы можем увидеть, что state dict содержит матрицы весов и смещений для каждого из наших слоёв.

In [ ]:
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

Простейшее действие - просто сохранить state dict с помощью `torch.save`. Например, мы можем сохранить его в файл `'checkpoint.pth'`.

In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')

Затем мы можем загрузить state dict с помощью `torch.load`.

In [ ]:
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

А чтобы загрузить state dict в сеть, нужно сделать `model.load_state_dict(state_dict)`.

In [ ]:
model.load_state_dict(state_dict)

Это кажется довольно простым, но, как обычно, это чуть сложнее. Загрузка state dict работает только если архитектура модели точно такая же, как архитектура чекпоинта (checkpoint). Если создать модель с другой архитектурой, это приведёт к ошибке.

In [ ]:
# Попробуем это сделать
model = fc_model.Network(784, 10, [400, 200, 100])
# Это вызовет ошибку, потому что размеры тензоров неверны!
model.load_state_dict(state_dict)

Это означает, что нам нужно точно восстановить модель так, как она была при обучении. Информация об архитектуре модели должна быть сохранена в контрольной точке вместе с state dict. Для этого вы создаёте словарь со всей информацией, необходимой для полного восстановления модели.

In [ ]:
checkpoint = {'input_size': 784,
              'output_size': 10,
              'hidden_layers': [each.out_features for each in model.hidden_layers],
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

Теперь контрольная точка содержит всю необходимую информацию для восстановления обученной модели. Вы можете легко сделать это функцией, если хотите. Аналогично, мы можем написать функцию для загрузки контрольных точек.

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [ ]:
model = load_checkpoint('checkpoint.pth')
print(model)